In [ ]:
# Exploring quantum sweep data
import os
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Define the directory containing the CSV files
csv_dir = os.path.join(os.getcwd(), 'quantum_sweep_results')

# Initialize a list to hold the dataframes
dataframes = []

# Read and process each CSV file in the directory
for csv_file in os.listdir(csv_dir):
    if csv_file.endswith('.csv'):
        csv_path = os.path.join(csv_dir, csv_file)
        tmp_df = pd.read_csv(csv_path)
        
        # Check if 'high_backoff' column is missing and process accordingly
        if 'high_backoff' not in tmp_df.columns:
            tmp_df['new_prediction_unscaled'] = tmp_df['new_prediction_unscaled'].apply(lambda x: x.strip('[]').split())
            floats = tmp_df['new_prediction_unscaled'].apply(lambda x: [float(num) for num in x])
            tmp_df['high_backoff'] = floats.apply(lambda x: x[0])
            tmp_df['medium_backoff'] = floats.apply(lambda x: x[1])
            tmp_df['low_backoff'] = floats.apply(lambda x: x[2])
            tmp_df.drop(columns=['new_prediction_unscaled'], inplace=True)
        
        dataframes.append(tmp_df)

# Concatenate all dataframes
df = pd.concat(dataframes)

# Remove rows with zero values in any of the backoff columns
df = df[(df[['high_backoff', 'medium_backoff', 'low_backoff']] != 0).all(axis=1)]

# Drop rows with NaN values in the specified columns
df.dropna(subset=['high_backoff', 'medium_backoff', 'low_backoff'], inplace=True)

# Calculate cosine similarity
cosine_sim = cosine_similarity(df[['high_backoff', 'medium_backoff', 'low_backoff']], np.array([[1.06, 3.8, 6.1]]))
df['cosine_sim'] = cosine_sim

# Calculate Euclidean distance
euclidean_dist = df.apply(lambda x: ((x['high_backoff'] - 1.06)**2 + (x['medium_backoff'] - 3.8)**2 + (x['low_backoff'] - 6.1)**2)**0.5, axis=1)
df['euclidean_dist'] = euclidean_dist

# Display the dataframe and its summary statistics
print(df)
print(df.describe())
print(df.columns)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Set the style and color palette
sns.set(style="whitegrid")
palette = sns.color_palette("tab10")

# Define a list of markers
markers = ['o', 's', 'D', '^', 'v', '<', '>', 'p', '*', 'h']

# Plot Euclidean distance vs. number of qubits grouped by batch size with different markers
plt.figure(figsize=(8, 8))
sns.lineplot(
    data=df, 
    x='n_wires', 
    y='euclidean_dist', 
    hue='batch_size', 
    style='batch_size', 
    markers=markers, 
    markersize=12, 
    palette=palette, 
    linewidth=2.5, 
    alpha=0.8, 
    errorbar=None
)

# Enhance plot
plt.title('Euclidean Distance vs. Number of Qubits Grouped by Batch Size', fontsize=18, pad=20)
plt.xlabel('Number of Qubits', fontsize=14, labelpad=15)
plt.ylabel('Euclidean Distance', fontsize=14, labelpad=15)
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.legend(title='Batch Size', fontsize=12, title_fontsize=14, loc='right', bbox_to_anchor=(1, 0.5))
plt.tight_layout()

# Save and show the plot
plt.savefig('qubit_count_euclidean_dist.png')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Set the style and color palette
sns.set(style="whitegrid")
palette = sns.color_palette("tab10")

# Define a list of markers
markers = ['o', 's', 'D', '^', 'v', '<', '>', 'p', '*', 'h']

# Plot Cosine Similarity vs. Number of Qubits grouped by Batch Size with different markers
plt.figure(figsize=(8, 8))
sns.lineplot(
    data=df, 
    x='n_wires', 
    y='cosine_sim', 
    hue='batch_size', 
    style='batch_size', 
    markers=markers, 
    markersize=10, 
    palette=palette, 
    linewidth=2.5, 
    alpha=0.8, 
    errorbar=None
)

# Enhance plot
plt.title('Cosine Similarity vs. Number of Qubits Grouped by Batch Size', fontsize=18, pad=20)
plt.xlabel('Number of Qubits', fontsize=14, labelpad=15)
plt.ylabel('Cosine Similarity', fontsize=14, labelpad=15)
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.legend(title='Batch Size', fontsize=12, title_fontsize=14, loc='right', bbox_to_anchor=(1, 0.5))
plt.tight_layout()

# Save and show the plot
plt.savefig('qubit_count_cos_similarity.png')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Pivot the data to prepare for heatmap
df_aggregated = df.groupby(['batch_size', 'n_wires'], as_index=False).mean()
euclidean_pivot = df_aggregated.pivot(index='batch_size', columns='n_wires', values='euclidean_dist')

# Set the style and color palette
sns.set(style="whitegrid")
palette = sns.color_palette("tab10")

# Create the heatmap
plt.figure(figsize=(8, 8))
sns.heatmap(euclidean_pivot, annot=True, cmap='Blues_r', fmt=".2f", linewidths=.5, annot_kws={"size": 20})

# Enhance plot
plt.title('Euclidean Distance across different Batch Sizes and Qubit Counts', fontsize=18, pad=20)
plt.xlabel('Number of Qubits', fontsize=18, labelpad=15)
plt.ylabel('Batch Size', fontsize=18, labelpad=15)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.tight_layout()

# Save and show the plot
plt.savefig('qubit_count_euclidean_distance_heatmap.png')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Aggregate duplicate entries by taking their mean
df_aggregated = df.groupby(['batch_size', 'n_wires'], as_index=False).mean()
cosine_pivot = df_aggregated.pivot(index='batch_size', columns='n_wires', values='cosine_sim')

# Set the style and color palette
sns.set(style="whitegrid")
palette = sns.color_palette("tab10")

# Create the heatmap
plt.figure(figsize=(8, 8))
sns.heatmap(cosine_pivot, annot=True, cmap='Blues', fmt=".2f", linewidths=.5, annot_kws={"size": 20})

# Enhance plot
plt.title('Cosine Similarity across Different Batch Sizes and Qubit Counts', fontsize=18, pad=20)
plt.xlabel('Number of Qubits', fontsize=18, labelpad=15)
plt.ylabel('Batch Size', fontsize=18, labelpad=15)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.tight_layout()

# Save and show the plot
plt.savefig('qubit_count_cos_similarity_heatmap.png')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Aggregate duplicate entries by taking their mean
df_aggregated = df.groupby(['batch_size', 'n_wires'], as_index=False).mean()

# Pivot the data to prepare for heatmaps
euclidean_pivot = df_aggregated.pivot(index='batch_size', columns='n_wires', values='euclidean_dist')
cosine_pivot = df_aggregated.pivot(index='batch_size', columns='n_wires', values='cosine_sim')

# Set the style and color palette
sns.set(style="whitegrid")
palette = sns.color_palette("tab10")

# Create the figure and subplots
fig, axes = plt.subplots(1, 2, figsize=(16, 8))

# Create the Euclidean Distance heatmap
sns.heatmap(euclidean_pivot, annot=True, cmap='Blues_r', fmt=".2f", linewidths=.5, annot_kws={"size": 20}, ax=axes[0])
axes[0].set_title('Euclidean Distance across Different Batch Sizes and Qubit Counts', fontsize=18, pad=20)
axes[0].set_xlabel('Number of Qubits', fontsize=18, labelpad=15)
axes[0].set_ylabel('Batch Size', fontsize=18, labelpad=15)
axes[0].tick_params(axis='x', labelsize=18)
axes[0].tick_params(axis='y', labelsize=18)

# Create the Cosine Similarity heatmap
sns.heatmap(cosine_pivot, annot=True, cmap='Blues', fmt=".2f", linewidths=.5, annot_kws={"size": 20}, ax=axes[1])
axes[1].set_title('Cosine Similarity across Different Batch Sizes and Qubit Counts', fontsize=18, pad=20)
axes[1].set_xlabel('Number of Qubits', fontsize=18, labelpad=15)
axes[1].set_ylabel('Batch Size', fontsize=18, labelpad=15)
axes[1].tick_params(axis='x', labelsize=18)
axes[1].tick_params(axis='y', labelsize=18)

# Adjust layout for better spacing
plt.tight_layout()

# Save and show the plot
plt.savefig('qubit_count_comparison_heatmaps.png')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Aggregate duplicate entries by taking their mean
df_aggregated = df.groupby(['batch_size', 'n_wires'], as_index=False).mean()

# Pivot the data to prepare for heatmaps
euclidean_pivot = df_aggregated.pivot(index='batch_size', columns='n_wires', values='euclidean_dist')
cosine_pivot = df_aggregated.pivot(index='batch_size', columns='n_wires', values='cosine_sim')

# Set the style and color palette
sns.set(style="whitegrid")
palette = sns.color_palette("tab10")

# Create the figure and subplots
fig, axes = plt.subplots(1, 2, figsize=(20, 6))

# Create the Euclidean Distance heatmap
sns.heatmap(euclidean_pivot, annot=True, cmap='Blues_r', fmt=".2f", linewidths=.5, annot_kws={"size": 20}, ax=axes[0])
axes[0].set_title('Euclidean Distance across Different Batch Sizes and Qubit Counts', fontsize=18, pad=20)
axes[0].set_xlabel('Number of Qubits', fontsize=18, labelpad=15)
axes[0].set_ylabel('Batch Size', fontsize=18, labelpad=15)
axes[0].tick_params(axis='x', labelsize=18)
axes[0].tick_params(axis='y', labelsize=18)

# Create the Cosine Similarity heatmap
sns.heatmap(cosine_pivot, annot=True, cmap='Blues', fmt=".2f", linewidths=.5, annot_kws={"size": 20}, ax=axes[1])
axes[1].set_title('Cosine Similarity across Different Batch Sizes and Qubit Counts', fontsize=18, pad=20)
axes[1].set_xlabel('Number of Qubits', fontsize=18, labelpad=15)
axes[1].set_ylabel('Batch Size', fontsize=18, labelpad=15)
axes[1].tick_params(axis='x', labelsize=18)
axes[1].tick_params(axis='y', labelsize=18)

# Adjust layout for better spacing
plt.tight_layout()

# Save and show the plot
plt.savefig('qubit_count_comparison_heatmaps.png')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Set the style and color palette
sns.set(style="whitegrid")
palette = sns.color_palette("tab10")

# Plot Cosine Similarity vs. batch size grouped by number of qubits
plt.figure(figsize=(12, 8))
sns.lineplot(
    data=df, 
    x='batch_size', 
    y='cosine_sim', 
    hue='n_wires', 
    marker='o', 
    palette=palette, 
    linewidth=2.5, 
    alpha=0.8, 
    errorbar=None
)

# Enhance plot
plt.title('Cosine Similarity vs. Batch Size Grouped by Number of Qubits', fontsize=18, pad=20)
plt.xlabel('Batch Size', fontsize=14, labelpad=15)
plt.ylabel('Cosine Similarity', fontsize=14, labelpad=15)
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.legend(title='Number of Qubits', fontsize=12, title_fontsize=14, loc='center left', bbox_to_anchor=(1, 0.5))
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Set the style and color palette
sns.set(style="whitegrid")
palette = sns.color_palette("tab10")

# Plot Cosine Similarity vs. batch size grouped by number of qubits
plt.figure(figsize=(12, 8))
sns.lineplot(
    data=df, 
    x='batch_size', 
    y='cosine_sim', 
    hue='n_wires', 
    marker='o', 
    palette=palette, 
    linewidth=2.5, 
    alpha=0.8, 
    errorbar=None
)

# Enhance plot
plt.title('Cosine Similarity vs. Batch Size Grouped by Number of Qubits', fontsize=18, pad=20)
plt.xlabel('Batch Size', fontsize=14, labelpad=15)
plt.ylabel('Cosine Similarity', fontsize=14, labelpad=15)
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.legend(title='Number of Qubits', fontsize=12, title_fontsize=14, loc='center left', bbox_to_anchor=(1, 0.5))
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Assuming df is already defined and contains 'n_wires', 'euclidean_dist', and 'n_blocks' columns
# Example DataFrame for reference
# df = pd.DataFrame({
#     'n_wires': [1, 2, 3, 4, 5, 1, 2, 3, 4, 5],
#     'euclidean_dist': [1.5, 2.3, 3.1, 4.0, 5.2, 1.2, 2.1, 2.9, 3.8, 4.5],
#     'n_blocks': [16, 16, 16, 16, 16, 32, 32, 32, 32, 32]
# })

# Filter the DataFrame to exclude points where n_blocks > 250
df_filtered = df[df['n_blocks'] <= 250]

# Set the style and color palette
sns.set(style="whitegrid")
palette = sns.color_palette("tab10")

# Plot Euclidean distance vs. n_blocks grouped by n_wires
plt.figure(figsize=(8, 8))
sns.lineplot(data=df_filtered, x='n_blocks', y='euclidean_dist', hue='n_wires', marker='o', palette=palette, linewidth=2.5, ci=None)

# Enhance plot
plt.title('Euclidean Distance vs. Number of Blocks Grouped by Number of Qubits', fontsize=18, pad=20)
plt.xlabel('Number of Blocks', fontsize=14, labelpad=15)
plt.ylabel('Euclidean Distance', fontsize=14, labelpad=15)
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.legend(title='Number of Qubits', fontsize=12, title_fontsize=14, loc='center left', bbox_to_anchor=(1, 0.5))
plt.tight_layout()

# Show the plot
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Assuming df is already defined and contains 'n_wires', 'training_time', 'batch_size', and 'euclidean_dist' columns
# Example DataFrame for reference
# df = pd.DataFrame({
#     'n_wires': [1, 2, 3, 4, 5, 1, 2, 3, 4, 5],
#     'training_time': [10, 20, 30, 40, 50, 15, 25, 35, 45, 55],
#     'batch_size': [32, 32, 32, 32, 32, 64, 64, 64, 64, 64],
#     'euclidean_dist': [1.5, 2.3, 3.1, 4.0, 5.2, 1.2, 2.1, 2.9, 3.8, 4.5]
# })

# Set the style and color palette
sns.set(style="whitegrid")
palette = sns.color_palette("tab10")

# Define a list of markers
markers = ['o', 's', 'D', '^', 'v', '<', '>', 'p', '*', 'h']

# Create a figure and a set of subplots
plt.figure(figsize=(8, 6))

# Plot training time vs. batch_size grouped by n_wires with different markers
plot = sns.lineplot(
    data=df, 
    x='batch_size', 
    y='training_time', 
    hue='n_wires', 
    style='n_wires', 
    markers=markers, 
    palette=palette, 
    linewidth=2.5, 
    errorbar=None, 
    markersize=10
)

# Enhance plot
plt.title('Training Time vs. Batch Size Grouped by Number of Qubits', fontsize=18, pad=20)
plt.xlabel('Batch Size', fontsize=14, labelpad=15)
plt.ylabel('Training Time (seconds)', fontsize=14, labelpad=15)
plt.grid(True, which='both', linestyle='--', linewidth=0.5)
plt.legend(title='Number of Qubits', fontsize=12, title_fontsize=14, loc='right', bbox_to_anchor=(1, 0.5))
plt.tight_layout()
plt.savefig('training_time_vs_batch_size.png')

# Show the plot
plt.show()

In [ ]:
import numpy as np

# Define functions for each metric

# 1. Mean Absolute Error (MAE)
def mean_absolute_error(x, y):
    return np.mean(np.abs(np.array(x) - np.array(y)))

# 2. Mean Squared Error (MSE)
def mean_squared_error(x, y):
    return np.mean((np.array(x) - np.array(y)) ** 2)

# 3. Cosine Similarity
def cosine_similarity(x, y):
    x, y = np.array(x), np.array(y)
    return np.dot(x, y) / (np.linalg.norm(x) * np.linalg.norm(y))

# 4. Euclidean Distance
def euclidean_distance(x, y):
    return np.linalg.norm(np.array(x) - np.array(y))

# 5. Pearson Correlation Coefficient
def pearson_correlation(x, y):
    return np.corrcoef(np.array(x), np.array(y))[0, 1]

# 6. Ratio of Means
def ratio_of_means(x, y):
    return np.mean(x) / np.mean(y)

# 7. Maximum Absolute Difference
def max_absolute_difference(x, y):
    return np.max(np.abs(np.array(x) - np.array(y)))

# 8. Manhattan Distance
def manhattan_distance(x, y):
    return np.sum(np.abs(np.array(x) - np.array(y)))

# Calculate Euclidean distance - for plotting
euclidean_dist_xy = np.linalg.norm(np.array(tnn) - np.array(qnn))

# Calculate Manhattan distance - for plotting
manhattan_dist_xy = np.sum(np.abs(np.array(tnn) - np.array(qnn)))

# Example input for testing
x = [1.06, 3.8, 6.1]  # Network 1 output
y = [1.2546277, 2.5873742, 5.028647]  # Network 2 output

# Test the functions
mae = mean_absolute_error(x, y)
mse = mean_squared_error(x, y)
cos_sim = cosine_similarity(x, y)
euclid_dist = euclidean_distance(x, y)
pearson_corr = pearson_correlation(x, y)
rom = ratio_of_means(x, y)
mad = max_absolute_difference(x, y)
manhat_dist = manhattan_distance(x, y)
print(f"Mean Absolute Error:            {mae:<20}")
print(f"Mean Squared Error:             {mse:<20}")
print(f"Cosine Similarity:              {cos_sim:<20}")
print(f"Euclidean Distance:             {euclid_dist:<20}")
print(f"Pearson Correlation Coefficient:{pearson_corr:<20}")
print(f"Ratio of Means:                 {rom:<20}")
print(f"Maximum Absolute Difference:    {mad:<20}")
print(f"Manhattan Distance:             {manhat_dist:<20}")


In [ ]:
def plot_3d_vectors(tnn, qnn, euclidean_dist, manhattan_dist):
        fig = plt.figure(figsize=(8, 8))
        ax = fig.add_subplot(111, projection='3d')

        # Plot points x and y in 3D
        ax.scatter(*tnn, color='blue', label='Traditional NN (1.06, 3.8, 6.1)')
        ax.scatter(*qnn, color='green', label='Quantum NN (1.25, 2.59, 5.03)')

        # Draw the Euclidean distance (straight line)
        ax.plot([tnn[0], qnn[0]], [tnn[1], qnn[1]], [tnn[2], qnn[2]],
                        color='red', linestyle='--', label=f'Euclidean Distance = {euclidean_dist:.2f}')

        # Annotate the points
        ax.text(tnn[0], tnn[1], tnn[2], f'TNN {tnn}', fontsize=10, color='blue', fontweight='bold', verticalalignment='top')
        ax.text(qnn[0], qnn[1], qnn[2], f'QNN {qnn}', fontsize=10, color='green', fontweight='bold', verticalalignment='top')

        # Set axis labels
        ax.set_xlabel('X-axis')
        ax.set_ylabel('Y-axis')
        ax.set_zlabel('Z-axis')

        # Add a legend and title
        ax.legend()
        ax.set_title('Euclidean Distance Visualization between Vectors')

        # Show the 3D plot
        plt.show()

# Example vectors for visualization
tnn = [1.06, 3.8, 6.1]  # Network 1 output
qnn = [1.2, 2.6, 5.0]  # Network 2 output
# Calculate Euclidean distance
euclidean_dist_xy = np.linalg.norm(np.array(tnn) - np.array(qnn))

# Calculate Manhattan distance
manhattan_dist_xy = np.sum(np.abs(np.array(tnn) - np.array(qnn)))

# Call the function
plot_3d_vectors(tnn, qnn, euclidean_dist_xy, manhattan_dist_xy)



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.patheffects as path_effects

def plot_3d_vectors(points1, points2):
    fig = plt.figure(figsize=(12, 8))
    ax = fig.add_subplot(111, projection='3d')

    # Set background color
    ax.set_facecolor('whitesmoke')

    # Ensure both lists have the same length
    assert len(points1) == len(points2), "Both point lists must have the same length."

    # Iterate through each pair of corresponding points
    for i, (tnn, qnn) in enumerate(zip(points1, points2)):
        # Calculate Euclidean distance
        euclidean_dist = np.linalg.norm(np.array(tnn) - np.array(qnn))

        # Plot points in 3D
        color = colors[i % len(colors)]
        ax.scatter(*tnn, color=color, s=150, marker='X', edgecolors='k')
        ax.scatter(*qnn, color=color, s=150, marker='o', edgecolors='k')

        # Draw the Euclidean distance (straight line)
        ax.plot([tnn[0], qnn[0]], [tnn[1], qnn[1]], [tnn[2], qnn[2]],
                color=color, linestyle='--', linewidth=2, label=f'Distance {i+1} = {euclidean_dist:.2f}')

        # Annotate the Euclidean distance on the line
        mid_point = [(tnn[0] + qnn[0]) / 2, (tnn[1] + qnn[1]) / 2, (tnn[2] + qnn[2]) / 2]
        text = ax.text(mid_point[0], mid_point[1], mid_point[2], f'{euclidean_dist:.2f}', fontsize=10, color=color, fontweight='bold', verticalalignment='top')
        text.set_path_effects([path_effects.Stroke(linewidth=3, foreground='white'), path_effects.Normal()])

    # Set axis labels
    ax.set_xlabel('X-axis', fontsize=14, labelpad=15)
    ax.set_ylabel('Y-axis', fontsize=14, labelpad=15)
    ax.set_zlabel('Z-axis', fontsize=14, labelpad=15)

    # Add a legend and title
    ax.legend(fontsize=10)
    ax.set_title('Euclidean Distance Visualization between\nTraditional and Quantum Neural Networks', fontsize=18, pad=20, loc='center', y=1.05)

    # Add grid lines with a darker color
    ax.grid(True, color='gray', linestyle='-', linewidth=0.5)

    # Show the 3D plot
    plt.show()

# Example lists of points for visualization
tnn_points = [
    [round(1.06, 2), round(3.8, 2), round(6.1, 2)],  # Point 1
    [round(0.9550018, 2), round(3.9018857, 2), round(6.2808137, 2)],  # Point 2
    [round(3.5326087, 2), round(5.93297, 2), round(6.9194074, 2)],  # Point 3
    [round(3.6497803, 2), round(3.7572162, 2), round(3.2952993, 2)],  # Point 4
    [round(6.1602077, 2), round(8.214929, 2), round(8.32826, 2)]   # Point 5
]

qnn_points = [
    [round(1.2, 2), round(2.6, 2), round(5.0, 2)],   # Point 1
    [round(2.5429058, 2), round(3.9086134, 2), round(6.2714934, 2)],  # Point 2
    [round(2.9910543, 2), round(3.966363, 2), round(5.976071, 2)],  # Point 3
    [round(4.8855886, 2), round(5.217516, 2), round(3.6067123, 2)],  # Point 4
    [round(2.430173, 2), round(4.082435, 2), round(6.1362176, 2)]   # Point 5
]

# Call the function
plot_3d_vectors(tnn_points, qnn_points)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.patheffects as path_effects

def plot_3d_vectors(points1, points2):
    fig = plt.figure(figsize=(12, 8))
    ax = fig.add_subplot(111, projection='3d')

    # Set background color to white
    ax.set_facecolor('white')

    # Ensure both lists have the same length
    assert len(points1) == len(points2), "Both point lists must have the same length."

    # Define a color palette using tab10
    colors = list(plt.cm.tab10.colors)

    # Define a list of linestyles
    linestyles = ['-', '--', '-.', ':']

    # Iterate through each pair of corresponding points
    for i, (tnn, qnn) in enumerate(zip(points1, points2)):
        # Calculate Euclidean distance
        euclidean_dist = np.linalg.norm(np.array(tnn) - np.array(qnn))

        # Plot points in 3D
        color = colors[i % len(colors)]
        linestyle = linestyles[i % len(linestyles)]
        ax.scatter(*tnn, color=color, s=150, marker='X', edgecolors='k')
        ax.scatter(*qnn, color=color, s=150, marker='o', edgecolors='k')

        # Draw the Euclidean distance (straight line)
        ax.plot([tnn[0], qnn[0]], [tnn[1], qnn[1]], [tnn[2], qnn[2]],
                color=color, linestyle=linestyle, linewidth=2, label=f'Distance {i+1} = {euclidean_dist:.2f}')

        # Annotate the Euclidean distance on the line
        mid_point = [(tnn[0] + qnn[0]) / 2, (tnn[1] + qnn[1]) / 2, (tnn[2] + qnn[2]) / 2]
        text = ax.text(mid_point[0], mid_point[1], mid_point[2], f'{euclidean_dist:.2f}', fontsize=10, color=color, fontweight='bold', verticalalignment='top')
        text.set_path_effects([path_effects.Stroke(linewidth=3, foreground='white'), path_effects.Normal()])

    # Set axis labels with darker colors
    ax.set_xlabel('X-axis', fontsize=14, labelpad=15, color='black')
    ax.set_ylabel('Y-axis', fontsize=14, labelpad=15, color='black')
    ax.set_zlabel('Z-axis', fontsize=14, labelpad=15, color='black')

    # Add a legend and title with darker colors
    ax.legend(fontsize=10)
    ax.set_title('Euclidean Distance Visualization between\nTraditional and Quantum Neural Networks', fontsize=18, pad=20, loc='center', y=1.05)

    # Add grid lines with a darker color
    ax.grid(True, color='gray', linestyle='-', linewidth=0.5)
    plt.savefig('euclidean_distance_example.png', bbox_inches='tight')
    # Show the 3D plot
    plt.show()

# Example lists of points for visualization
tnn_points = [
    [round(1.06, 2), round(3.8, 2), round(6.1, 2)],  # Point 1
    [round(3.5326087, 2), round(5.93297, 2), round(6.9194074, 2)],  # Point 3
    [round(3.6497803, 2), round(3.7572162, 2), round(3.2952993, 2)],  # Point 4
    [round(6.1602077, 2), round(8.214929, 2), round(8.32826, 2)]   # Point 5
]

qnn_points = [
    [round(1.2, 2), round(2.6, 2), round(5.0, 2)],   # Point 1
    [round(2.9910543, 2), round(3.966363, 2), round(5.976071, 2)],  # Point 3
    [round(4.8855886, 2), round(5.217516, 2), round(3.6067123, 2)],  # Point 4
    [round(2.430173, 2), round(4.082435, 2), round(6.1362176, 2)]   # Point 5
]

# Call the function
plot_3d_vectors(tnn_points, qnn_points)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.patheffects as path_effects

def plot_cosine_similarity(vectors1, vectors2):
    """
    Plot cosine similarity between two lists of 3D vectors.
    
    Parameters:
    vectors1: list of 3D vectors (first set of vectors)
    vectors2: list of 3D vectors (second set of vectors)
    """
    fig = plt.figure(figsize=(8, 8))
    ax = fig.add_subplot(111, projection='3d')

    # Get the tab20 colormap
    tab20_colors = plt.get_cmap('tab10').colors

    # Function to calculate cosine similarity
    def cosine_similarity(vec1, vec2):
        dot_product = np.dot(vec1, vec2)
        norm_vec1 = np.linalg.norm(vec1)
        norm_vec2 = np.linalg.norm(vec2)
        return dot_product / (norm_vec1 * norm_vec2)

    legend_entries = []

    for i, (v1, v2) in enumerate(zip(vectors1, vectors2)):
        # Normalize the vectors
        v1_norm = v1 / np.linalg.norm(v1)
        v2_norm = v2 / np.linalg.norm(v2)
        
        # Assign a unique color to each pair
        color = tab20_colors[i % len(tab20_colors)]
        
        # Plot the vectors
        ax.quiver(0, 0, 0, v1_norm[0], v1_norm[1], v1_norm[2], color=color, alpha=0.6)
        ax.quiver(0, 0, 0, v2_norm[0], v2_norm[1], v2_norm[2], color=color, alpha=0.6, linestyle='dashed')
        
        # Calculate cosine similarity
        cos_sim = cosine_similarity(v1, v2)
        
        # Annotate the cosine similarity at the midpoint between the vectors
        mid_point = [(v1_norm[0] + v2_norm[0]) / 2, (v1_norm[1] + v2_norm[1]) / 2, (v1_norm[2] + v2_norm[2]) / 2]
        text = ax.text(mid_point[0], mid_point[1], mid_point[2], f'{cos_sim:.2f}', fontsize=10, color='black', fontweight='bold', verticalalignment='top')
        text.set_path_effects([path_effects.Stroke(linewidth=3, foreground='white'), path_effects.Normal()])

        # Add to legend entries
        legend_entries.append((color, f'Pair {i+1}: Cosine Similarity = {cos_sim:.2f}'))

    # Set plot labels and limits
    ax.set_xlim([0, 0.75])
    ax.set_ylim([0, 0.75])
    ax.set_zlim([0, 0.75])
    ax.set_xlabel('X-axis')
    ax.set_ylabel('Y-axis')
    ax.set_zlabel('Z-axis', labelpad=0)
    ax.set_title('Cosine Similarity between Traditional and Quantum Neural Network predictions')

    # Create custom legend
    custom_lines = [plt.Line2D([0], [0], color=color, lw=4) for color, _ in legend_entries]
    legend_labels = [label for _, label in legend_entries]
    ax.legend(custom_lines, legend_labels, loc='upper right')

    plt.show()

tnn_points = [
    [round(1.06, 2), round(3.8, 2), round(6.1, 2)],  # Point 1
    [round(3.5326087, 2), round(5.93297, 2), round(6.9194074, 2)],  # Point 2
    [round(3.6497803, 2), round(3.7572162, 2), round(3.2952993, 2)],  # Point 3
    [round(6.1602077, 2), round(8.214929, 2), round(8.32826, 2)]   # Point 4
]

qnn_points = [
    [round(1.2, 2), round(2.6, 2), round(5.0, 2)],   # Point 1
    [round(2.9910543, 2), round(3.966363, 2), round(5.976071, 2)],  # Point 2
    [round(4.8855886, 2), round(5.217516, 2), round(3.6067123, 2)],  # Point 3
    [round(2.430173, 2), round(4.082435, 2), round(6.1362176, 2)]   # Point 4
]
# Call the function to plot cosine similarity
plot_cosine_similarity(tnn_points, qnn_points)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Updated TNN and QNN points as provided by the user
tnn_points = [
    [round(1.06, 2), round(3.8, 2), round(6.1, 2)],  # Point 1
    [round(3.5326087, 2), round(5.93297, 2), round(6.9194074, 2)],  # Point 2
    [round(3.6497803, 2), round(3.7572162, 2), round(3.2952993, 2)],  # Point 3
    [round(6.1602077, 2), round(8.214929, 2), round(8.32826, 2)]   # Point 4
]

qnn_points = [
    [round(1.2, 2), round(2.6, 2), round(5.0, 2)],   # Point 1
    [round(2.9910543, 2), round(3.966363, 2), round(5.976071, 2)],  # Point 2
    [round(4.8855886, 2), round(5.217516, 2), round(3.6067123, 2)],  # Point 3
    [round(2.430173, 2), round(4.082435, 2), round(6.1362176, 2)]   # Point 4
]

# Assuming cosine similarities for the new points
cosine_similarities = [1.00, 0.99, 1.00, 0.98]

# Create subplots of radar charts for each pair of TNN and QNN points with cosine similarity in the titles
fig, axs = plt.subplots(2, 2, figsize=(12, 12), subplot_kw=dict(polar=True))

labels = ['X-axis', 'Y-axis', 'Z-axis']
angles = np.linspace(0, 2 * np.pi, len(labels), endpoint=False).tolist()
angles += angles[:1]

pairs = [
    ("TNN 1", "QNN 1"),
    ("TNN 2", "QNN 2"),
    ("TNN 3", "QNN 3"),
    ("TNN 4", "QNN 4")
]
titles = ["Cosine Similarity 1",
          "Cosine Similarity 2",
          "Cosine Similarity 3",
          "Cosine Similarity 4"]
# Define colors for each graph
tab10_colors = plt.cm.tab10.colors
colors = [tab10_colors[0], tab10_colors[1], tab10_colors[2], tab10_colors[3]]

# Iterate through the pairs and create separate radar charts in each subplot, adding cosine similarity to the title
for i, ax in enumerate(axs.flatten()):
    tnn, qnn = tnn_points[i], qnn_points[i]
    data_tnn = np.concatenate([tnn, [tnn[0]]])
    data_qnn = np.concatenate([qnn, [qnn[0]]])
    
    # Plot each pair
    ax.plot(angles, data_tnn, linewidth=2, label=pairs[i][0], linestyle='--', color=colors[i])
    ax.plot(angles, data_qnn, linewidth=2, label=pairs[i][1], linestyle='-', color=colors[i])
    ax.fill(angles, data_qnn, alpha=0.25, color=colors[i])
    
    ax.set_ylim(0, 9)
    ax.set_xticks(angles[:-1])
    ax.set_xticklabels(labels, fontsize=18)
    ax.set_title(f"{titles[i]}: {cosine_similarities[i]:.2f}", fontsize=22)
    ax.legend(loc='upper right', fontsize=14)

# Adjust layout for better spacing
plt.tight_layout()
plt.savefig('cosine_similarity_example.png')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.patheffects as path_effects
from sklearn.metrics.pairwise import cosine_similarity

def plot_3d_vectors(points1, points2):
    fig = plt.figure(figsize=(12, 8))
    ax = fig.add_subplot(111, projection='3d')

    # Set background color to white
    ax.set_facecolor('white')

    # Ensure both lists have the same length
    assert len(points1) == len(points2), "Both point lists must have the same length."

    # Define a color palette using tab10
    colors = list(plt.cm.tab10.colors)

    # Define a list of linestyles
    linestyles = ['-', '--', '-.', ':']

    # Iterate through each pair of corresponding points
    for i, (tnn, qnn) in enumerate(zip(points1, points2)):
        # Calculate Euclidean distance
        euclidean_dist = np.linalg.norm(np.array(tnn) - np.array(qnn))

        # Plot points in 3D
        color = colors[i % len(colors)]
        linestyle = linestyles[i % len(linestyles)]
        ax.scatter(*tnn, color=color, s=150, marker='X', edgecolors='k')
        ax.scatter(*qnn, color=color, s=150, marker='o', edgecolors='k')

        # Draw the Euclidean distance (straight line)
        ax.plot([tnn[0], qnn[0]], [tnn[1], qnn[1]], [tnn[2], qnn[2]],
                color=color, linestyle=linestyle, linewidth=2, label=f'Distance {i+1} = {euclidean_dist:.2f}')

        # Annotate the Euclidean distance on the line
        mid_point = [(tnn[0] + qnn[0]) / 2, (tnn[1] + qnn[1]) / 2, (tnn[2] + qnn[2]) / 2]
        text = ax.text(mid_point[0], mid_point[1], mid_point[2], f'{euclidean_dist:.2f}', fontsize=10, color=color, fontweight='bold', verticalalignment='top')
        text.set_path_effects([path_effects.Stroke(linewidth=3, foreground='white'), path_effects.Normal()])

    # Set axis labels with darker colors
    ax.set_xlabel('X-axis', fontsize=14, labelpad=15, color='black')
    ax.set_ylabel('Y-axis', fontsize=14, labelpad=15, color='black')
    ax.set_zlabel('Z-axis', fontsize=14, labelpad=15, color='black')

    # Add a legend and title with darker colors
    ax.legend(fontsize=10)
    ax.set_title('Euclidean Distance Visualization between\nTraditional and Quantum Neural Networks', fontsize=18, pad=20, loc='center', y=1.05)

    # Add grid lines with a darker color
    ax.grid(True, color='gray', linestyle='-', linewidth=0.5)
    plt.savefig('euclidean_distance_example.png', bbox_inches='tight')
    # Show the 3D plot
    plt.show()



def calculate_cosine_similarities(tnn_points, qnn_points):
    assert len(tnn_points) == len(qnn_points), "Both point lists must have the same length."
    
    cosine_similarities = []
    for tnn, qnn in zip(tnn_points, qnn_points):
        # Reshape the points to 2D arrays as required by cosine_similarity
        tnn = np.array(tnn).reshape(1, -1)
        qnn = np.array(qnn).reshape(1, -1)
        
        # Calculate cosine similarity and append to the list
        similarity = cosine_similarity(tnn, qnn)[0][0]
        cosine_similarities.append(similarity)
    
    return cosine_similarities

cosine_similarities = calculate_cosine_similarities(tnn_points, qnn_points)
print(cosine_similarities)

def plot_cosine_similarity(tnn, qnn, cosine_similarities):
    # Create subplots of radar charts for each pair of TNN and QNN points with cosine similarity in the titles
    fig, axs = plt.subplots(2, 2, figsize=(12, 12), subplot_kw=dict(polar=True))

    labels = ['X-axis', 'Y-axis', 'Z-axis']
    angles = np.linspace(0, 2 * np.pi, len(labels), endpoint=False).tolist()
    angles += angles[:1]

    pairs = [
        ("TNN 1", "QNN 1"),
        ("TNN 2", "QNN 2"),
        ("TNN 3", "QNN 3"),
        ("TNN 4", "QNN 4")
    ]
    titles = ["Cosine Similarity 1",
            "Cosine Similarity 2",
            "Cosine Similarity 3",
            "Cosine Similarity 4"]
    # Define colors for each graph
    tab10_colors = plt.cm.tab10.colors
    colors = [tab10_colors[0], tab10_colors[1], tab10_colors[2], tab10_colors[3]]

    # Iterate through the pairs and create separate radar charts in each subplot, adding cosine similarity to the title
    for i, ax in enumerate(axs.flatten()):
        tnn, qnn = tnn_points[i], qnn_points[i]
        data_tnn = np.concatenate([tnn, [tnn[0]]])
        data_qnn = np.concatenate([qnn, [qnn[0]]])
        
        # Plot each pair
        ax.plot(angles, data_tnn, linewidth=2, label=pairs[i][0], linestyle='--', color=colors[i])
        ax.plot(angles, data_qnn, linewidth=2, label=pairs[i][1], linestyle='-', color=colors[i])
        ax.fill(angles, data_qnn, alpha=0.25, color=colors[i])
        
        ax.set_ylim(0, 9)
        ax.set_xticks(angles[:-1])
        ax.set_xticklabels(labels, fontsize=18)
        ax.set_title(f"{titles[i]}: {cosine_similarities[i]:.2f}", fontsize=22)
        ax.legend(loc='upper right', fontsize=14)

    # Adjust layout for better spacing
    plt.tight_layout()
    plt.savefig('cosine_similarity_example.png')
    plt.show()



# Example lists of points for visualization
tnn_points = [
    [round(1.06, 2), round(3.8, 2), round(6.1, 2)],  # Point 1
    [round(3.5326087, 2), round(5.93297, 2), round(6.9194074, 2)],  # Point 2
    [round(3.6497803, 2), round(3.7572162, 2), round(3.2952993, 2)],  # Point 3
    [round(6.1602077, 2), round(8.214929, 2), round(8.32826, 2)]   # Point 4
]

qnn_points = [
    [round(1.2, 2), round(2.6, 2), round(5.0, 2)],   # Point 1
    [round(2.9910543, 2), round(3.966363, 2), round(5.976071, 2)],  # Point 2
    [round(4.8855886, 2), round(5.217516, 2), round(3.6067123, 2)],  # Point 3
    [round(2.430173, 2), round(4.082435, 2), round(6.1362176, 2)]   # Point 4
]

plot_3d_vectors(tnn_points, qnn_points)
plot_cosine_similarity(tnn_points, qnn_points, cosine_similarities)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.patheffects as path_effects
from sklearn.metrics.pairwise import cosine_similarity
from matplotlib.gridspec import GridSpec

def plot_combined(tnn_points, qnn_points, cosine_similarities):
    fig = plt.figure(figsize=(24, 12))
    gs = GridSpec(1, 2, width_ratios=[1, 1.5], figure=fig)

    # First main subplot: 3D vectors
    ax1 = fig.add_subplot(gs[0], projection='3d')
    colors = list(plt.cm.tab10.colors)
    linestyles = ['-', '--', '-.', ':']
    for i, (tnn, qnn) in enumerate(zip(tnn_points, qnn_points)):
        euclidean_dist = np.linalg.norm(np.array(tnn) - np.array(qnn))
        color = colors[i % len(colors)]
        linestyle = linestyles[i % len(linestyles)]
        ax1.scatter(*tnn, color=color, s=150, marker='X', edgecolors='k')
        ax1.scatter(*qnn, color=color, s=150, marker='o', edgecolors='k')
        ax1.plot([tnn[0], qnn[0]], [tnn[1], qnn[1]], [tnn[2], qnn[2]], color=color, linestyle=linestyle, linewidth=2, label=f'Pair {i+1} Distance = {euclidean_dist:.2f}')
        mid_point = [(tnn[0] + qnn[0]) / 2, (tnn[1] + qnn[1]) / 2, (tnn[2] + qnn[2]) / 2]
        text = ax1.text(mid_point[0], mid_point[1], mid_point[2], f'{euclidean_dist:.2f}', fontsize=10, color=color, fontweight='bold', verticalalignment='top')
        text.set_path_effects([path_effects.Stroke(linewidth=3, foreground='white'), path_effects.Normal()])
    ax1.set_xlabel('X-axis', fontsize=14, labelpad=15, color='black')
    ax1.set_ylabel('Y-axis', fontsize=14, labelpad=15, color='black')
    ax1.set_zlabel('Z-axis', fontsize=14, labelpad=15, color='black')
    ax1.legend(fontsize=10)
    ax1.set_title('Euclidean Distance Visualization between\nTraditional and Quantum Neural Networks', fontsize=18, pad=20, loc='center', y=1.05)
    ax1.grid(True, color='gray', linestyle='-', linewidth=0.5)

    # Second main subplot: Radar charts
    gs2 = gs[1].subgridspec(2, 2)
    labels = ['X-axis', 'Y-axis', 'Z-axis']
    angles = np.linspace(0, 2 * np.pi, len(labels), endpoint=False).tolist()
    angles += angles[:1]

    pairs = [
        ("TNN 1", "QNN 1"),
        ("TNN 2", "QNN 2"),
        ("TNN 3", "QNN 3"),
        ("TNN 4", "QNN 4")
    ]
    titles = ["Pair 1 cosine similarity",
              "Pair 2 cosine similarity",
              "Pair 3 cosine similarity",
              "Pair 4 cosine similarity"]
    tab10_colors = plt.cm.tab10.colors
    colors = [tab10_colors[0], tab10_colors[1], tab10_colors[2], tab10_colors[3]]

    for i in range(4):
        ax = fig.add_subplot(gs2[i], polar=True)
        tnn, qnn = tnn_points[i], qnn_points[i]
        data_tnn = np.concatenate([tnn, [tnn[0]]])
        data_qnn = np.concatenate([qnn, [qnn[0]]])
        ax.plot(angles, data_tnn, linewidth=2, label=pairs[i][0], linestyle='--', color=colors[i])
        ax.plot(angles, data_qnn, linewidth=2, label=pairs[i][1], linestyle='-', color=colors[i])
        ax.fill(angles, data_qnn, alpha=0.25, color=colors[i])
        ax.set_ylim(0, 9)
        ax.set_xticks(angles[:-1])
        ax.set_xticklabels(labels, fontsize=18)
        ax.set_title(f"{titles[i]}: {cosine_similarities[i]:.2f}", fontsize=22)
        ax.legend(loc='upper right', fontsize=14)

    plt.tight_layout()
    plt.savefig('combined_plot_euclidean_cosine_plot.png')
    plt.show()

def calculate_cosine_similarities(tnn_points, qnn_points):
    assert len(tnn_points) == len(qnn_points), "Both point lists must have the same length."
    
    cosine_similarities = []
    for tnn, qnn in zip(tnn_points, qnn_points):
        tnn = np.array(tnn).reshape(1, -1)
        qnn = np.array(qnn).reshape(1, -1)
        similarity = cosine_similarity(tnn, qnn)[0][0]
        cosine_similarities.append(similarity)
    
    return cosine_similarities

# Example lists of points for visualization
tnn_points = [
    [round(1.06, 2), round(3.8, 2), round(6.1, 2)],  # Point 1
    [round(3.5326087, 2), round(5.93297, 2), round(6.9194074, 2)],  # Point 2
    [round(3.6497803, 2), round(3.7572162, 2), round(3.2952993, 2)],  # Point 3
    [round(6.1602077, 2), round(8.214929, 2), round(8.32826, 2)]   # Point 4
]

qnn_points = [
    [round(1.2, 2), round(2.6, 2), round(5.0, 2)],   # Point 1
    [round(2.9910543, 2), round(3.966363, 2), round(5.976071, 2)],  # Point 2
    [round(4.8855886, 2), round(5.217516, 2), round(3.6067123, 2)],  # Point 3
    [round(2.430173, 2), round(4.082435, 2), round(6.1362176, 2)]   # Point 4
]

cosine_similarities = calculate_cosine_similarities(tnn_points, qnn_points)
plot_combined(tnn_points, qnn_points, cosine_similarities)

In [ ]:
import numpy as np

probability_requested = [
    [.4, .2, .1],
    [0.3, 0.2, 0.05],
   # [0.4, 0.3, 0.1],
    [0.2, 0.2, 0.2],
    [0.6, 0.3, 0.1],
]

# Example lists of points for visualization
tnn_points = [
    [2, 4, 6],
    [round(1.06, 2), round(3.8, 2), round(6.1, 2)],  # Point 1
    [round(3.5326087, 2), round(5.93297, 2), round(6.9194074, 2)],  # Point 2
    [round(3.6497803, 2), round(3.7572162, 2), round(3.2952993, 2)],  # Point 3
    [round(6.1602077, 2), round(8.214929, 2), round(8.32826, 2)]   # Point 4
]

qnn_points = [
    [4, 8, 12],
    [round(1.2, 2), round(2.6, 2), round(5.0, 2)],   # Point 1
    [round(2.9910543, 2), round(3.966363, 2), round(5.976071, 2)],  # Point 2
    [round(4.8855886, 2), round(5.217516, 2), round(3.6067123, 2)],  # Point 3
    [round(2.430173, 2), round(4.082435, 2), round(6.1362176, 2)]   # Point 4
]
# Calculate and print Euclidean distances and cosine similarities
for prob, tnn, qnn in zip(probability_requested, tnn_points, qnn_points):
    euclidean_dist = np.linalg.norm(np.array(tnn) - np.array(qnn))
    cosine_sim = np.dot(np.array(tnn), np.array(qnn)) / (np.linalg.norm(np.array(tnn)) * np.linalg.norm(np.array(qnn)))
    print(f'Target Probability: {prob}, TNN Point: {tnn}, QNN Point: {qnn}, Euclidean Distance: {euclidean_dist:.2f}, Cosine Similarity: {cosine_sim:.2f}')
